In [ ]:
#Kmeans clustering segmentation

import numpy as np
import pandas as pd
import cv2
import collections
import seaborn as sns
import matplotlib.pyplot as plt
from skimage import filters
from skimage import util
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import MeanShift, estimate_bandwidth
import warnings
warnings.filterwarnings("ignore")

In [ ]:
imageName = 'Datasets/Retina/Stare/Images/im0001.ppm'
image = plt.imread(imageName)
plt.figure(dpi=150)
plt.title('EYE')
plt.imshow(image)

In [ ]:
index = pd.MultiIndex.from_product(
    (*map(range, image.shape[:2]), ('r', 'g', 'b')),
    names=('row', 'col', None))
df_1 = pd.Series(image.flatten(), index=index)
df_1 = df_1.unstack()
df_1 = df_1.reset_index().reindex(columns=['col','row',   'r','g','b'])
df_1.head(10)

In [ ]:
df_2 = df_1[['r','g','g']]
df_2.head(10)

In [ ]:
nd_1 = df_1.values.astype(np.float32)
nd_1[:, 0] = nd_1[:, 0] / (nd_1[:, 0].max() + 1)
nd_1[:, 1] = nd_1[:, 1] / (nd_1[:, 1].max() + 1)
nd_1[:, 2:] = nd_1[:, 2:] / 255
nd_2 = df_2.astype(np.float64) / 255

In [ ]:
nd_1 = MinMaxScaler(feature_range=(0, 1)).fit_transform(df_1)
nd_2 = MinMaxScaler(feature_range=(0, 1)).fit_transform(df_2)

In [ ]:
total_inertias_1 = [KMeans(n_clusters=i).fit(nd_1).inertia_ for i in range(1, 20)]
total_inertias_2 = [KMeans(n_clusters=i).fit(nd_2).inertia_ for i in range(1, 20)]
print(total_inertias_1)
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, sharex='col', sharey='row', figsize=(30,10))
ax1.plot(range(1, 20), total_inertias_1, c='black')
ax1.set(xlabel='Total Inertias', ylabel='Number of Clusters', title='RGB Pixels with locations')
ax2.plot(range(1, 20), total_inertias_2, c='black')
ax2.set(xlabel='Total Inertias', title='RGB Pixels without locations');
plt.savefig('elbow_result.jpg')

In [ ]:
#mean shift

In [ ]:
bandwidth_1 = estimate_bandwidth(nd_1, quantile=.04, n_jobs=-1)
bandwidth_2 = estimate_bandwidth(nd_2, quantile=.04, n_jobs=-1);

In [ ]:
ms_1 = MeanShift(bandwidth = bandwidth_1 , n_jobs=-1, bin_seeding=True, cluster_all=True).fit(nd_1)
ms_2 = MeanShift(bandwidth = bandwidth_2 , n_jobs=-1, bin_seeding=True, cluster_all=True).fit(nd_2)

In [ ]:
print(ms_1.cluster_centers_[:, 2:])
print(ms_1.labels_)

img = np.array(image)
shape = img.shape

labels1 = ms_1.labels_
labels2 = ms_2.labels_
result_image1 = np.reshape(labels1, shape[:2])
result_image2 = np.reshape(labels2, shape[:2])

plt.figure(1)
plt.clf()
plt.axis('off')
plt.title('Original image', loc='center')
plt.imshow(image)
plt.figure(2)
plt.clf()
plt.axis('off')
plt.title('Pixels with their location image ({} colors, Mean-Shift)'.format(len(ms_1.cluster_centers_)), loc='center')
# plt.imshow(recreate_image(ms_1.cluster_centers_[:, 2:], ms_1.labels_, width, height))
plt.imshow(result_image1) 
plt.figure(3)
plt.clf()
plt.axis('off')
plt.title('Pixels without their location ({} colors, Mean-Shift)'.format(len(ms_2.cluster_centers_)), loc='center')
# plt.imshow(recreate_image(ms_2.cluster_centers_, ms_2.labels_, width, height))
plt.imshow(result_image2) 